In [4]:
from DoD import column_infer
from DoD.view_search_pruning import ViewSearchPruning
from knowledgerepr import fieldnetwork
from modelstore.elasticstore import StoreHandler

model_path = "/Users/gongyue/aurum-datadiscovery/test/advwModels/"
output_path = "/Users/gongyue/aurum-datadiscovery/test/advwResult/"
sep = ","

store_client = StoreHandler()
network = fieldnetwork.deserialize_network(model_path)
columnInfer = column_infer.ColumnInfer(network=network, store_client=store_client, csv_separator=sep)
viewSearch = ViewSearchPruning(network=network, store_client=store_client, csv_separator=sep)

In [5]:
attrs = ["", "", ""]
values = [["Amy", "Alberts", "European Sales Manager"],
            ["Ryan", "Cornelsen", "Production Technician - WC40"],
            ["Gary", "Altman", "Facilities Manager"]]
types = ["", ""]
column_clusters = columnInfer.get_clusters(attrs, values, types)

col_values = {}
for (idx, clusters) in enumerate(column_clusters):
    col_values[clusters[0]["name"]] = [row[idx] for row in values]

In [6]:
from DoD.utils import FilterType

import ipywidgets as widgets
from IPython.display import display, clear_output

output = widgets.Output()
c_id = 0
filter_drs = {}

@output.capture()
def initial_show():
    columnInfer.show_clusters(column_clusters[0], filter_drs, viewSearch, 0)
    
@output.capture()
def on_button_next(b):
    global c_id
    clear_output()
    c_id = min(len(column_clusters)-1, c_id + 1)
    columnInfer.show_clusters(column_clusters[c_id], filter_drs, viewSearch, c_id)
    
@output.capture()
def on_button_prev(b):
    global c_id
    clear_output()
    c_id = max(0, c_id - 1)
    columnInfer.show_clusters(column_clusters[c_id], filter_drs, viewSearch, c_id)

button_next = widgets.Button(description="Next")
button_prev = widgets.Button(description="Prev")
display(widgets.HBox([button_prev, button_next]))
button_next.on_click(on_button_next)
button_prev.on_click(on_button_prev)
initial_show()
display(output)

Output()

In [7]:
from DoD import data_processing_utils as dpu

view_metadata_mapping = dict()
view_id = 0
perf_stats = dict()
k = 10
output = widgets.Output()

views = []
for mjp, attrs_project, metadata, join_path in viewSearch.virtual_schema_iterative_search(col_values, filter_drs, perf_stats, max_hops=2,
                                                                               debug_enumerate_all_jps=False, offset=k):
        proj_view = dpu.project(mjp, attrs_project)
        views.append((proj_view, join_path))
        
@output.capture()
def initial_show_view():
    display(views[0][0])
    print(views[0][1])
    
@output.capture()
def on_button_next_view(b):
    global view_id
    clear_output()
    view_id = min(len(views)-1, view_id + 1)
    display(views[view_id][0])
    print(views[view_id][1])
    
@output.capture()
def on_button_prev_view(b):
    global view_id
    clear_output()
    view_id = max(0, view_id - 1)
    display(views[view_id][0])
    print(views[view_id][1])
    
button_next_2 = widgets.Button(description="Next")
button_prev_2 = widgets.Button(description="Prev")
display(widgets.HBox([button_prev_2, button_next_2]))
button_next_2.on_click(on_button_next_view)
button_prev_2.on_click(on_button_prev_view)
initial_show_view()
display(output)
print("total views:", len(views))


Candidate group: ['Person.csv', 'Employee.csv']
Covers #Filters: 3

Finding paths between Person.csv and Employee.csv
max hops: 2
Total time: 0.004813194274902344

Candidate group: ['Address.csv', 'Employee.csv']
Covers #Filters: 3

Finding paths between Address.csv and Employee.csv
max hops: 2
Total time: 0.009408950805664062
Finished enumerating groups


Output()

total views: 2
